# Log文件解析脚本
 
目前，已经可以从Log中解析以下表格中的内容：  

|类型|正则名称|数据名称|  
|:-|:-|:-|
|MCLoc  |mcloc_re|t, x, y, theta, confidence|  
|Yaw|yaw_re|yaw_t, yaw_data, yaw_time, ax, ay, az, gx, gy, gz, offx, offy, offz|  
|Odometer|odometer_re|odo_t, odo_time, odo_x, odo_y, odo_theta|  
|下发速度|vel_re|vx, vy, omega|  
|TEB下发速度|send_vel_re|send_vel_t, send_vx, send_vy, send_vw, send_steer_angle|  
|TEB获取速度|get_vel_re|get_vel_t, get_vel_x, get_vel_vw, get_vel_steer_angle|  

##  如何添加新的解析内容
1. [正则模块中添加新的正则表达式和对应的变量](#No1)  
2. [解析函数中处理解析出来的数据，赋予对应变量](#No2)  
3. [运行解析函数，并且画图](#No3)

In [1]:
%matplotlib qt5
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import math
from datetime import datetime

## <span id="No1">正则 </span>

* 定位

In [2]:
mcloc_re = re.compile("\[(.*?)\]\[MCLoc\].*x:(.*?):y:(.*?):theta:(.*?):confidence:(.*?)$")
t = []
x = []
y = []
theta =[]
confidence = []

 * 陀螺仪

In [3]:
yaw_re = re.compile("\[(.*?)\].*yaw: (.*?),? ?timestamp: (\d+)")
yaw_t = []
yaw_data = []
yaw_time = []
acc_re = re.compile("acc = \[(.*?), (.*?), (.*?)\]w = \[(.*?), (.*?), (.*?)\];off = \[(.*?), (.*?), (.*?)\]")
ax = []
ay = []
az = []
gx = []
gy = []
gz = []
offx = []
offy = []
offz = []

* 里程

In [4]:
odometer_re = re.compile("\[(.*?)\].*timestamp: (\d+), x: (.*?), y: (.*?), angle: (.*?),")
odo_t = []
odo_time = []
odo_x = []
odo_y = []
odo_theta = []

* 下发速度

In [5]:
vel_re = re.compile('\[(.*?)\].*{ "vx" : (.*?), "vy" : (.*?), "w" : (.*?) }')
vel_t = []
vx =[]
vy = []
omega = []

* TEB中下发速度

In [6]:
send_vel_re = re.compile('\[(.*?)\].* vx = (.*?); vy = (.*?); vw = (.*?); steer_angle = (.*?); max_vel = .*')
send_vel_t = []
send_vx = []
send_vy = []
send_vw = []
send_steer_angle = []

* TEB获取速度

In [21]:
get_vel_re = re.compile('\[(.*?)\].* vx = (.*?); omega = (.*?); steer_angle = (.*?)\n')
get_vel_t = []
get_vx = []
get_vw = []
get_steer_angle = []

## <span id="No2">解析函数 </span>

In [141]:
def rbktimetodate(rbktime):
    return datetime.strptime(rbktime, '%Y-%m-%d %H:%M:%S.%f')
def findrange(ts, t1, t2):
    small_ind = -1
    large_ind = len(ts)-1
    for i, data in enumerate(ts):
        large_ind = i
        if(t1 < data and small_ind < 0):
            small_ind = i
        if(t2 < data):
            break;
    return small_ind, large_ind
def anglejoint(anglelist, period = 360.0):
    base_offset = anglelist[0]
    anglelist[0] = 0
    noise = 0.01 * 360
    print(base_offset)
    for i in range(1, len(anglelist)):
        temp = anglelist[i] - base_offset
        if(temp - anglelist[i - 1] > (period-noise)):
            anglelist[i] =temp - period
        elif (temp - anglelist[i - 1] < (-period+noise)):
            anglelist[i] =temp + period
        else:
            anglelist[i] = temp
        if(anglelist[i] > period/2.0):
            anglelist[i] -= period
        elif(anglelist[i] < -period/2.0):
            anglelist[i] += period
def read_log(filename): 
    line_num = 0
    for line in open(filename): 
        line_num += 1
        #print(line_num)
        m = mcloc_re.match(line)
        yaw = yaw_re.match(line)
        odo = odometer_re.match(line)
        acc = acc_re.match(line)
        vel = vel_re.match(line)
        send_vel = send_vel_re.match(line)
        get_vel = get_vel_re.match(line)
        if m:
            t.append(rbktimetodate(m.group(1)))
            x.append(float(m.group(2)))
            y.append(float(m.group(3)))
            theta.append(float(m.group(4)))
            confidence.append(float(m.group(5)))
        if yaw:
            yaw_t.append(rbktimetodate(yaw.group(1)))
            yaw_data.append((float(yaw.group(2))/math.pi *  180.0))
            yaw_time.append(float(yaw.group(3)))
        if acc:
            ax.append(float(acc.group(1)))
            ay.append(float(acc.group(2)))
            az.append(float(acc.group(3)))
            gx.append(float(acc.group(4))/math.pi*180.0*16.4)
            gy.append(float(acc.group(5))/math.pi*180.0*16.4)
            gz.append(float(acc.group(6))/math.pi*180.0*16.4)
            offx.append(float(acc.group(7)))
            offy.append(float(acc.group(8)))
            offz.append(float(acc.group(9)))
        if odo:
            odo_t.append(rbktimetodate(odo.group(1)))
            odo_time.append(float(odo.group(2)))
            odo_x.append(float(odo.group(3)))
            odo_y.append(float(odo.group(4)))
            odo_theta.append((float(odo.group(5))/math.pi * 180.0))
        if vel:
            vel_t.append(rbktimetodate(vel.group(1)))
            vx.append(float(vel.group(2)))
            vy.append(float(vel.group(3)))
            omega.append(float(vel.group(4)))
        if send_vel:
            send_vel_t.append(rbktimetodate(send_vel.group(1)))
            send_vx.append(float(send_vel.group(2)))
            send_vy.append(float(send_vel.group(3)))
            send_vw.append(float(send_vel.group(4)))
            send_steer_angle.append(float(send_vel.group(5)))
        if get_vel:
            get_vel_t.append(rbktimetodate(get_vel.group(1)))
            get_vx.append(float(get_vel.group(2)))
            get_vw.append(float(get_vel.group(3)))
            get_steer_angle.append(float(get_vel.group(4)))

## <span id="No3">解析文件，并画图 </span>

In [148]:
filename= 'robokit_2012-01-11_20-08-47.843_v2.log'
#filename = 'F:\\dazu\\robokit_2012-01-11_13-44-13.827\\robokit_2012-01-11_13-44-13.827.log'
t, x, y, theta, confidence = [], [], [], [], []
yaw_t, yaw_data, yaw_time = [], [], []
ax, ay, az = [], [], []
gx, gy, gz = [], [], []
offx, offy, offz = [], [], []
odo_t, odo_time, odo_x, odo_y, odo_theta = [], [], [], [], []
vel_t, vx, vy, omega = [], [], [], []
send_vel_t, send_vx, send_vy, send_vw, send_steer_angle = [], [], [], [], []
get_vel_t, get_vx, get_vw, get_steer_angle = [], [], [], []
read_log(filename)
anglejoint(theta)
anglejoint(yaw_data)
anglejoint(odo_theta)

-30.528
59.89987269195192
93.66198372783082


In [153]:
plt.figure(1)
plt.plot(t,confidence)

In [151]:

plt.figure(2)
plt.plot(t,theta,'g--',label='mcloc ')
plt.plot(odo_t,odo_theta,'r--',label='Odo ')
plt.plot(yaw_t,yaw_data,'k--',label='Gyro ')
plt.legend()

In [150]:
fig = plt.figure(6)
ax1 = fig.add_subplot(211)
ax1.plot(t,x,'b-',label='x')
ax1.plot(t,y,'r-',label='y')
ax1.legend(loc = 'upper right')
ax2 = ax1.twinx()
ax2.plot(send_vel_t,send_vx,'g.',label='auto vx')
ax2.plot(send_vel_t,send_vw,'y.',label='auto omega')
ax2.legend(loc = 'upper left')
ax1 = fig.add_subplot(212)
ax1.plot(t,y,'r-',label='y')

In [129]:
t1 = rbktimetodate("2018-02-10 14:29:07.0")
t2 = rbktimetodate("2018-02-10 14:29:26.0")
ind1, ind2 = findrange(t, t1, t2)
plt.figure(2)
plt.subplot(3,1,1)
plt.title('Go Strght Directly')
plt.plot(t[ind1:ind2],theta[ind1:ind2],'g--',label='mcloc ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(odo_t, t1, t2)
plt.subplot(3,1,2)
plt.plot(odo_t[ind1:ind2],odo_theta[ind1:ind2],'r--',label='Odo ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(yaw_t, t1, t2)
plt.subplot(3,1,3)
plt.plot(yaw_t[ind1:ind2],yaw_data[ind1:ind2],'k--',label='Gyro ')
plt.legend(loc = 'upper right')


In [127]:
t1 = rbktimetodate("2018-02-10 14:24:21.0")
t2 = rbktimetodate("2018-02-10 14:26:06.0")
ind1, ind2 = findrange(t, t1, t2)
plt.figure(3)
plt.subplot(3,1,1)
plt.title('Go Strght DWA')
plt.plot(t[ind1:ind2],theta[ind1:ind2],'g--',label='mcloc ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(odo_t, t1, t2)
plt.subplot(3,1,2)
plt.plot(odo_t[ind1:ind2],odo_theta[ind1:ind2],'r--',label='Odo ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(yaw_t, t1, t2)
plt.subplot(3,1,3)
plt.plot(yaw_t[ind1:ind2],yaw_data[ind1:ind2],'k--',label='Gyro ')
plt.legend(loc = 'upper right')

In [128]:
t1 = rbktimetodate("2018-02-10 14:26:26.0")
t2 = rbktimetodate("2018-02-10 14:28:04.0")
ind1, ind2 = findrange(t, t1, t2)
plt.figure(4)
plt.subplot(3,1,1)
plt.title('Go Strght DWA')
plt.plot(t[ind1:ind2],theta[ind1:ind2],'g--',label='mcloc ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(odo_t, t1, t2)
plt.subplot(3,1,2)
plt.plot(odo_t[ind1:ind2],odo_theta[ind1:ind2],'r--',label='Odo ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(yaw_t, t1, t2)
plt.subplot(3,1,3)
plt.plot(yaw_t[ind1:ind2],yaw_data[ind1:ind2],'k--',label='Gyro ')
plt.legend(loc = 'upper right')

In [34]:
fig = plt.figure(5)
ax1 = fig.add_subplot(311)
ax1.plot(t,theta,'g--',label='mcloc ')
ax1.plot(odo_t,odo_theta,'r--',label='Odo ')
ax1.plot(yaw_t,yaw_data,'k--',label='Gyro ')
ax1.legend()
ax2 = fig.add_subplot(312)
ax2.plot(yaw_t,gx,'b-',label='gx')
ax2.plot(yaw_t,gy,'r-',label='gy')
ax2.plot(yaw_t,gz,'g-',label='gz')
ax2.legend(loc = 'upper right')
ax3 = fig.add_subplot(313)
ax3.plot(yaw_t,offx,'b-',label='offx')
ax3.plot(yaw_t,offy,'r-',label='offy')
ax3.plot(yaw_t,offz,'g-',label='offz')

In [ ]:
fig = plt.figure(4)
ax1 = fig.add_subplot(311)
ax1.plot(t,theta,'g--',label='mcloc ')
ax1.plot(odo_t,odo_theta,'r--',label='Odo ')
ax1.plot(yaw_t,yaw_data,'k--',label='Gyro ')
ax1.legend()
ax2 = fig.add_subplot(312)
ax2.plot(yaw_t,gx,'b-',label='gx')
ax2.plot(yaw_t,gy,'r-',label='gy')
ax2.plot(yaw_t,gz,'g-',label='gz')
ax2.legend(loc = 'upper right')
ax3 = fig.add_subplot(313)
ax3.plot(yaw_t,offx,'b-',label='offx')
ax3.plot(yaw_t,offy,'r-',label='offy')
ax3.plot(yaw_t,offz,'g-',label='offz')

In [58]:
print(len(vel_t))

0


In [27]:
0.087/math.pi*180

4.984732817638162

In [33]:
0.001/math.pi *180 * 16.4

0.93965078401455

In [70]:
math.fabs(1.1)

1.1

In [102]:
t1 = rbktimetodate("2018-02-10 14:29:07.0")
t2 = rbktimetodate("2018-02-10 14:29:26.0")
ind1, ind2 = findrange(t, t1, t2)
print(ind1, ind2)

13407 13619
